# Despliegue

- Cargamos el modelo
- Cargamos los datos futuros
- Preparar los datos futuros
- Aplicamos el modelo para la predicción

In [48]:
#Importamos librerías básicas
import pandas as pd # manipulacion dataframes
import numpy as np  # matrices y vectores
import matplotlib.pyplot as plt #gráfica

In [49]:
#cargamos el pipeline
import pickle
filename = 'pipeline_modelo.pkl'
pipeline = pickle.load(open(filename, 'rb'))
pipeline


Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(transformers=[('cat',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='most_frequent')),
                                                                                   ('dummies',
                                                                                    OneHotEncoder(drop='if_binary',
                                                                                                  handle_unknown='ignore',
                                                                                                  sparse_output=False))]),
                                                                   ['age',
                                                                    'physical_activity_minutes_per_week',
                                                                    'family_history_diabetes',
                                                                    'bmi',
                                                                    'waist_to_hip_ratio',
                                                                    'systolic_bp',
                                                                    'hdl_cholesterol',
                                                                    'ldl_cholesterol',
                                                                    'triglycerides',
                                                                    'glucose_fasting',
                                                                    'insulin_level',
                                                                    'hba1c'])]))])),
                ('model',
                 DecisionTreeClassifier(criterion='entropy', max_depth=8,
                                        min_samples_leaf=183,
                                        random_state=42))])

In [50]:
#Cargamos los datos futuros
#data = pd.read_excel("Diabetes-datosFuturos.xlsx")
#data.head()

,age,physical_activity_minutes_per_week,family_history_diabetes,bmi,waist_to_hip_ratio,systolic_bp,hdl_cholesterol,ldl_cholesterol,triglycerides,glucose_fasting,insulin_level,hba1c
0,Adulto Mayor,Adecuada,0,Normal,Bajo,Normal,Normal,Casi Óptimo,Normal,Normal,Normal,Normal
1,Adulto Medio,Adecuada,0,Obesidad,Moderado,Elevada,Normal,Óptimo,Límite Alto,Normal,Elevado,Normal
2,Adulto Mayor,Baja,0,Sobrepeso,Moderado,Hipertensión 1,Normal,Casi Óptimo,Normal,Prediabetes,Normal,Diabetes
3,Adulto Joven,Adecuada,0,Sobrepeso,Moderado,Normal,Normal,Óptimo,Normal,Prediabetes,Normal,Prediabetes
4,Anciano,Inactivo,0,Normal,Moderado,Hipertensión 2,Normal,Casi Óptimo,Normal,Prediabetes,Normal,Diabetes


In [28]:
#Se crea interfaz gráfica con streamlit para captura de los datos
#Interfaz gráfica
import streamlit as st
import pandas as pd
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.lib import colors

# Configuración de página
st.set_page_config(page_title="Predicción de Diabetes", page_icon="🩺", layout="centered")

st.title("Predicción de Diagnóstico de Diabetes")
st.write("Complete los datos del paciente para obtener una predicción.")


# OPCIONES DEL FORMULARIO
age_options = ['Joven', 'Adulto Joven', 'Adulto Medio', 'Adulto Mayor', 'Anciano']
physical_activity_options = ['Inactivo', 'Baja', 'Adecuada', 'Alta']
family_history_options = ['Si', 'No']  # Usuario ve SI/NO
bmi_options = ['Bajo Peso', 'Normal', 'Sobrepeso', 'Obesidad']
waist_hip_options = ['Bajo', 'Moderado', 'Alto']
systolic_bp_options = ['Normal', 'Elevada', 'Hipertensión 1', 'Hipertensión 2']
hdl_options = ['Bajo', 'Normal', 'Alto']
ldl_options = ['Óptimo', 'Casi Óptimo', 'Límite Alto', 'Alto', 'Muy Alto']
triglycerides_options = ['Normal', 'Límite Alto', 'Alto']
glucose_fasting_options = ['Normal', 'Prediabetes', 'Diabetes']
insulin_options = ['Bajo', 'Normal', 'Elevado', 'Muy Elevado']
hba1c_options = ['Normal', 'Prediabetes', 'Diabetes']


# FORMULARIO DE ENTRADA
st.subheader("Datos del Paciente")
col1, col2 = st.columns(2)

with col1:
    age = st.selectbox("Edad", age_options)
    physical_activity = st.selectbox("Actividad física semanal", physical_activity_options)
    family_history_diabetes = st.selectbox("Historial familiar de diabetes", family_history_options)
    bmi = st.selectbox("Índice de Masa Corporal (BMI)", bmi_options)
    waist_to_hip_ratio = st.selectbox("Relación cintura–cadera", waist_hip_options)
    systolic_bp = st.selectbox("Presión arterial sistólica", systolic_bp_options)

with col2:
    hdl_cholesterol = st.selectbox("HDL colesterol", hdl_options)
    ldl_cholesterol = st.selectbox("LDL colesterol", ldl_options)
    triglycerides = st.selectbox("Triglicéridos", triglycerides_options)
    glucose_fasting = st.selectbox("Glucosa en ayunas", glucose_fasting_options)
    insulin_level = st.selectbox("Nivel de insulina", insulin_options)
    hba1c = st.selectbox("HbA1c", hba1c_options)

st.markdown("---")

# CONVERSIÓN Si / No → 1 / 0
family_history_num = 1 if family_history_diabetes == "Si" else 0

# ARMAR DATAFRAME
data = pd.DataFrame([[
    age,
    physical_activity,
    family_history_num,
    bmi,
    waist_to_hip_ratio,
    systolic_bp,
    hdl_cholesterol,
    ldl_cholesterol,
    triglycerides,
    glucose_fasting,
    insulin_level,
    hba1c
]], columns=[
    'age',
    'physical_activity_minutes_per_week',
    'family_history_diabetes',
    'bmi',
    'waist_to_hip_ratio',
    'systolic_bp',
    'hdl_cholesterol',
    'ldl_cholesterol',
    'triglycerides',
    'glucose_fasting',
    'insulin_level',
    'hba1c'
])

# Mostrar dataframe al usuario PERO con Si/No
data_display = data.copy()
data_display['family_history_diabetes'] = data_display['family_history_diabetes'].map({1: "Si", 0: "No"})

st.subheader("Datos ingresados")
st.dataframe(data_display)

# BOTÓN DE PREDICIÓN
if st.button("Realizar predicción"):
    try:
        # Hacemos la predicción
        Y_fut = pipeline.predict(data)[0]

        # Convertimos predicción a nombre
        diagnostico = "No_Diabetes" if Y_fut == 0 else "SI_Diabetes"

        #COLORES EN PANTALLA
        if diagnostico == "No_Diabetes":
            st.markdown(
                "<h3 style='color:green;'>Diagnóstico del modelo: NO DIABETES</h3>",
                unsafe_allow_html=True
            )
        else:
            st.markdown(
                "<h3 style='color:red;'>Diagnóstico del modelo: SÍ DIABETES</h3>",
                unsafe_allow_html=True
            )

        # Agregar al dataframe mostrado
        data_display['Diagnostico_Prediccion'] = diagnostico

        st.subheader("Resultado final")
        st.dataframe(data_display)


        # GENERAR PDF
        pdf_filename = "resultado_diagnostico.pdf"
        doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
        styles = getSampleStyleSheet()
        elements = []

        title = "<b>Reporte de Predicción de Diabetes</b>"
        elements.append(Paragraph(title, styles['Title']))
        elements.append(Spacer(1, 0.3 * inch))

        # Datos del paciente
        elements.append(Paragraph("<b>Datos ingresados:</b>", styles['Heading2']))
        for col in data_display.columns:
            val = data_display[col].values[0]
            elements.append(Paragraph(f"<b>{col}:</b> {val}", styles['Normal']))

        elements.append(Spacer(1, 0.3 * inch))

        # Diagnóstico
        color_text = "green" if diagnostico == "No_Diabetes" else "red"
        diag_html = f"<b>Diagnóstico del Modelo:</b> <font color='{color_text}'>{diagnostico}</font>"
        elements.append(Paragraph(diag_html, styles['Heading2']))

        # Guardar PDF
        doc.build(elements)

        # Botón para descargar
        with open(pdf_filename, "rb") as f:
            st.download_button(
                label="Descargar resultado en PDF",
                data=f,
                file_name=pdf_filename,
                mime="application/pdf"
            )

    except Exception as e:
        st.error("Error realizando la predicción. Revisa el pipeline.")
        st.error(str(e))

ModuleNotFoundError: No module named 'streamlit'